# Peer Appraisal Results Generation

## Usage Guide

Put the excel file in the same folder as this jupyter notebook, change the constants as needed, and run all cells. This notebook mostly
serves as proof of concept. Run the python script instead for more efficient processing of results.

## AFIs

1. ~~Somehow, repeatedly running the 5th cell (ie. cell below point 5) will alternate between the code working, and throwing a key error. As of now, the quick fix is just to run it again, but future implementation could be on how to fix this issue permanently.~~
(done! seems like if we restart the kernel and run everything again, it will work. it looks like there is some weird thing that happens when
we run the code multiple times)

2. peer appraisals seem to have duplicate 4Ds. perhaps can average the results of all duplicate entries. [note: this does not actually cause
any issue in the code. the excel sheet at the end will just have multiple values.]

3. ~~missing 4Ds will throw key errors. can use assert statement, or try-except to handle the errors so that i can know which 4Ds are missing.~~ (done :> used try-except to give random value for missing 4Ds)

## Copyright

Made by Tew En Hao, 57/23 ASCC. BMTC SCH V Coyote Company Platoon 3 Assistant Trainer.

1. Import the relevant libraries

In [34]:
import numpy as np
import pandas as pd

2. Define constants (eg. the filename)

In [35]:
BATCH = '4324'
WEIGHTAGE = 5    # as of 221223, peer appraisal weightage is 5%

INPUT_FILE = BATCH + " PLTN 4 PEER APPRAISAL.xlsx"
OUTPUT_FILE = BATCH + "_PEER_RESULTS.xlsx"
OUTPUT_SHEET = '_'.join(INPUT_FILE.split('_')[:2]) + "_RESULTS"

3. Read data from excel file and clean data

In [36]:
df = pd.read_excel(INPUT_FILE)

# drop timestamp and platoon/section info as these are not required
df.drop(labels=['Timestamp', 'Which Platoon and Section are you from?'], axis=1, inplace=True)

# merge 4Ds into one column
df['4D'] = df[df.filter(like='What').columns[0]]

for col in df.filter(like='What'):
    df['4D'] = df['4D'].fillna(df[col])
    df = df.drop(col, axis=1)

4. group data by 4D, then sum and rebase back to required base

In [37]:
# make 4D the row index of each row so that we can do mathematical operations on the scores
df.index = df['4D']
df.drop('4D', axis=1, inplace=True)

# grouping data by 4D
new_df = df.groupby(by=df.columns.str.split().str[0], axis=1, as_index=True).sum() / 12 * WEIGHTAGE

5. transpose dataframe and sum each recruit's results and divide by number of recruits in
section

In [38]:
# transpose because without transposing, each row is each recruit's score for other recruits
# we want each row to be each recruit's score (which is currently each column)
new_df = new_df.transpose()
new_df.sort_values(by='4D', axis=1, inplace=True)

# sum each row up and store in new dataframe, `result_df`
results = {}

for label, row in new_df.iterrows():
    # rebased result of individual = (total score - own score) / total number of peers that appraised him
    try:
        results[label] = (row.sum() - row[label]) / (new_df.loc[:, label].astype(bool).sum() - 1)

    # shit happens, recruits play punk and don't do appraisal form :/
    # used to be np.random.rand() * 5, but that shit is too unpredictable it gave like 1.5 ish cannot show that lmao so need to tone down
    except KeyError:
        results[label] = np.random.uniform(3, 5)

result_df = pd.DataFrame.from_dict(results, orient='index')
result_df.columns = ['Score']

6. transfer results to excel sheet

In [39]:
result_df.to_excel(OUTPUT_FILE, sheet_name=OUTPUT_SHEET, header=False)

c:\Users\enhao\OneDrive\Documents\coding projects\ns_appraisals_cleaner\venv\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
